### Visualization Library Setup

In [ ]:
from IPython.display import display, Javascript, HTML
import json

In [ ]:
%%javascript
require.config({
    paths: {
        d3: "https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min"
     }
});

In [ ]:
Javascript(filename='tree.js')

In [ ]:
HTML("""
<style>

.node {
  cursor: pointer;
}

.node circle {
  fill: #d49c9c;
  stroke: #8c6666;
  stroke-width: 1.5px;
}

.node text {
  font-size: 11px !important;
  font-family: sans-serif;
  fill: #4545b5;
}

.link {
  fill: none;
  stroke: #efcece;
  stroke: #efceed;
  stroke-width: 2px;
}

.templink {
  fill: none;
  stroke: red;
  stroke-width: 2px;
}
</style>
""")


## NMODL Python Interface Tutorial

### Introduction

NMODL is a code generation framework for NEURON Modeling Language. It is primarily designed to support optimised code generation backends for morphologically detailed neuron simulators. It provides high level Python interface that can be used for model introspection as well as performing various analysis on underlying model.

The main goals of the NMODL framework are :

* Support for full NMODL specification
* Providing modular tools for lexing, parsing and analysis
* Optimised code generation for modern architectures
* Compatibility with exisiting simulators
* Ability to implement new simulator backends with minimal efforts

This tutorial provides introduction to python API with examples.

### Installation 
<a id='installation'></a> NMODL can be installed as CMake project or using python setuptools. See README.md for detailed installation instructions. For example :

```bash
python3 -m venv myenv
. myenv/bin/activate
pip3 install nmodl_source_directory/
```

With this you should have nmodl installed.

### Parsing Model And Constructing AST

Once the NMODL is setup properly we should be able to import `nmodl` module :

In [ ]:
import nmodl.dsl as nmodl

If you see any issues, check the [installation section](#installation). Lets take an example of a channelCaDynamics :

In [ ]:
channel = """
NEURON  {
    SUFFIX CaDynamics
    USEION ca READ ica WRITE cai
    RANGE decay, gamma, minCai, depth
}

UNITS   {
    (mV) = (millivolt)
    (mA) = (milliamp)
    FARADAY = (faraday) (coulombs)
    (molar) = (1/liter)
    (mM) = (millimolar)
    (um)    = (micron)
}

PARAMETER   {
    gamma = 0.05 : percent of free calcium (not buffered)
    decay = 80 (ms) : rate of removal of calcium
    depth = 0.1 (um) : depth of shell
    minCai = 1e-4 (mM)
}

ASSIGNED    {ica (mA/cm2)}

INITIAL {
    cai = minCai
}

STATE   {
    cai (mM)
}

BREAKPOINT  { SOLVE states METHOD cnexp }

DERIVATIVE states   {
    cai' = -(10000)*(ica*gamma/(2*FARADAY*depth)) - (cai - minCai)/decay
}

FUNCTION foo() {
    LOCAL temp
    foo = 1.0 + gamma
}
"""

Now we can parse any valid NMODL constructs using parsing interface. First, we have to create nmodl parser object using `nmodl::Driver` and then we can use `parse_string` method <a id='create-ast'></a>:

In [ ]:
driver = nmodl.Driver()
driver.parse_string(channel)

The `parse_string` method will throw an exception with parsing error if input is invalid. Otherwise it returns `True` and internally creates [AST](https://en.wikipedia.org/wiki/Abstract_syntax_tree) object. We can access the AST using `ast()` method :

In [ ]:
modast = driver.ast()

If we simply print AST object, we can see JSON representation :

In [ ]:
print ('%.100s' % modast)  # only first 100 characters
import json
json_data = json.loads(nmodl.to_json(modast, True))
json_data_expand = json.loads(nmodl.to_json(modast, True, True))

In [ ]:
Javascript("""(function(element){
                require(['draw_tree'], function(draw) { draw(element.get(0), %s) });
           })(element);""" % json.dumps(json_data_expand) )

### Querying AST object with Visitors

One of the strength of NMODL python interface is access to inbuilt [Visitors](https://en.wikipedia.org/wiki/Visitor_pattern). One can perform different queries and analysis on AST using different visitors. Lets start with the examples of inbuilt visitors.

#### Lookup visitor

As name suggest, lookup visitor allows to search different NMODL constructs in the AST. The `visitor` module provides access to inbuilt visitors. In order to use this visitor, we create an object of AstLookupVisitor :

In [ ]:
from nmodl.dsl import visitor
from nmodl.dsl import ast
lookup_visitor = visitor.AstLookupVisitor()

Assuming we have created AST object (as shown [here](#create-ast)), we can search for any NMODL construct in the AST using AstLookupVisitor. For example, to find out `STATE` block in the mod file, we can simply do:

In [ ]:
states = lookup_visitor.lookup(modast, ast.AstNodeType.STATE_BLOCK)
for state in states:
    print (nmodl.to_nmodl(state))

We have used `to_nmodl` helper function to print AST object in NMODL format. Note that the output of `to_nmodl` should be same as input except for comments (?, :, COMMENT). There are very few edge cases where the NMODL output could slightly differ and this is considered as bug. This is being addressed by testing entire [ModelDB](https://senselab.med.yale.edu/modeldb/) database.

Using AstLookupVisitor we can introspect NMODL constructs at any level of details. Below are some examples to find out different constructs in the example mod file:

In [ ]:
odes = lookup_visitor.lookup(modast, ast.AstNodeType.DIFF_EQ_EXPRESSION)
primes = lookup_visitor.lookup(modast, ast.AstNodeType.PRIME_NAME)
range_vars = lookup_visitor.lookup(modast, ast.AstNodeType.RANGE_VAR)
parameters = lookup_visitor.lookup(modast, ast.AstNodeType.PARAM_ASSIGN)
units = lookup_visitor.lookup(modast, ast.AstNodeType.UNIT)

if odes:
    print("%d differential equation(s) exist : " % len(odes))
    for ode in odes:
        print ("\t %s " % nmodl.to_nmodl(ode))
    
if primes:
    print("%d prime variables exist :" % len(primes), end='')
    for prime in primes:
        print (" %s" % nmodl.to_nmodl(prime), end='')
    print()

if range_vars:
    print("%d range variables exist :" % len(range_vars), end='')
    for range_var in range_vars:
        print (" %s" % nmodl.to_nmodl(range_var), end='')
    print()

if parameters:
    print("%d parameter variables exist :" % len(parameters), end='')
    for range_var in range_vars:
        print (" %s" % nmodl.to_nmodl(range_var), end='')
    print()
    
if units:
    print("%d units uses :" % len(units), end='')
    for unit in units:
        print (" %s" % nmodl.to_nmodl(unit), end='')

Apart from performing lookup on whole AST object (i.e. entire NMODL file), we can perform analysis on the specific construct. Lets take a synthetic example : say we want to find out all assignment statements in function `foo`. If we use lookup visitor on AST, it will returnn all statements in the mod file (e.g. including DERIVATIVE block). To avoid this, we can first find FUNCTION blocks and then search for statements within that block :

In [ ]:
functions = lookup_visitor.lookup(modast, ast.AstNodeType.FUNCTION_BLOCK)
function = functions[0] # first function

# expression statements include assignments
new_lookup_visitor = visitor.AstLookupVisitor(ast.AstNodeType.EXPRESSION_STATEMENT)

# using accept method of node
function.accept(new_lookup_visitor)
statements = new_lookup_visitor.get_nodes()

for statement in statements:
    print (nmodl.to_nmodl(statement))

Every AST node provides `accept` method that takes visitor object as parameter. In above example, we used `accept` method of `FunctionBock` node. This allows to run a given visitor on a specific node. `AstLookupVisitor` provides `get_nodes()` method that can be used to retrive the result of visitor. List of all `AstNodeType` can be found [here (todo)](####).

#### Symbol Table Visitor

Symbol table visitor is used to find out all variables and their usage in mod file. To use this, first create a visitor object as: 

In [ ]:
from nmodl.dsl import symtab

symv = symtab.SymtabVisitor()

Once the visitor object is created, we can run visitor on AST object to populate symbol table. Symbol table provides print method that can be used to print whole symbol table : 

In [ ]:
symv.visit_program(modast)
table = modast.get_symbol_table()
table_s = str(table)

print (table_s)

Now we can query for variables in the symbol table based on name of variable: 

In [ ]:
cai = table.lookup('cai')
print (cai)

When we print the symbol, all it's properties get printed. For example, in above case the `cai` variable is used in :
* differential equation (prime)
* state block
* assigned block
* use ion statement

We can also query based on the kind of variables. For example, to find out all range variables we can use `get_variables_with_properties` method with symbol property as an argument:

In [ ]:
range_vars = table.get_variables_with_properties(symtab.NmodlType.range_var)
for var in range_vars:
    print (var)

We can also query with  multiple properties. For example, to find out read or write ion variables we can use (second argument `False` indicates any one property):

In [ ]:
ions_var = table.get_variables_with_properties(symtab.NmodlType.read_ion_var | symtab.NmodlType.write_ion_var, False)
for var in ions_var:
    print (var)

#### Custom AST visitor

If predefined visitors are limited, we can implement new visitor using AstVisitor interface. Lets say we want to implement a visitor that will print every floating point number in MOD file. Here is how we can do this: 

In [ ]:
from nmodl.dsl import ast, visitor

class DoubleVisitor(visitor.AstVisitor):

    def visit_double(self, node):
        print (node.eval())  # or, can use nmodl.to_nmodl(node) 
        
d_visitor = DoubleVisitor()
modast.accept(d_visitor)

The `AstVisitor` base class provides all necessary methods to traverse different ast nodes. New visitors will inherit from `AstVisitor` and implement only those method where we want different behaviour. For example, in the above case we want to visit ast nodes of type `Double` and print their value. To achieve this we implemented associated method of `Double` node i.e. `visit_double`. When we call `accept` method on the ast object, the entire AST tree will be visited (by `AstVisitor`). But whenever double node type will encounter in AST, the control will be handed back to `DoubleVisitor.visit_double` method. 

Lets implement the example of lookup visitor to print parameters with values :

In [ ]:
class ParameterVisitor(visitor.AstVisitor):
    
    def __init__(self):
        visitor.AstVisitor.__init__(self)
        self.in_parameter = False
        
    def visit_param_block(self, node):
        self.in_parameter = True
        node.visit_children(self)
        self.in_parameter = False

    def visit_name(self, node):
        if self.in_parameter:
            print (nmodl.to_nmodl(node))
            
    def visit_double(self, node):
        if self.in_parameter:
            print (node.eval())

    def visit_integer(self, node):
        if self.in_parameter:
            print (node.eval())

param_visitor = ParameterVisitor()
modast.accept(param_visitor)